In [18]:
import numpy as np
import pandas as pd
import random

In [106]:
vectors_df = pd.read_csv('vectors.txt', delim_whitespace=True, header=None)
parsed_col = []
for keyword in list(vectors_df[0]):
    keyword = str(keyword)
    keyword = keyword.replace('_', ' ')
    keyword = keyword.replace('’', "'")
    parsed_col.append(keyword)
vectors_df[0] = parsed_col
vectors_df

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,300
0,singapore,0.483909,0.906052,-0.081405,-0.402945,-0.047337,-0.211383,0.105737,-0.323959,0.510133,...,-0.014059,0.151803,0.018653,0.178135,-0.288335,0.176272,0.748743,0.934583,-0.303212,0.107761
1,place,0.983583,0.080516,0.307405,-0.478417,-0.265591,0.097866,0.006518,-0.024012,0.867993,...,-0.764314,-0.474266,0.451464,0.648060,0.041233,0.448002,0.597665,0.580291,-0.613324,0.685953
2,gardens,0.631589,0.493373,0.344579,0.329813,-0.687817,-0.231521,-0.636084,0.556939,-0.330538,...,-0.464227,-0.003085,0.771097,-0.186565,-0.653929,-0.132298,0.597235,0.534219,0.756995,-0.097779
3,time,1.287348,0.745675,0.804061,-0.626885,0.316215,-0.374367,0.264915,-0.753390,-0.060295,...,0.337025,-1.014939,-0.187284,0.281302,-0.819577,-0.510459,0.313020,0.121017,-0.042820,-0.470455
4,night,0.571266,0.875816,-0.591916,-0.490323,-0.734806,0.068555,0.439737,0.373520,0.375961,...,-0.240506,-0.104857,-0.364867,-0.497946,-0.469053,-0.853215,0.371460,-0.077505,0.404010,0.513231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20644,zoo site,-0.075530,-0.020636,0.014292,0.036398,0.023679,0.046482,0.109715,0.034369,-0.073305,...,0.008565,0.019394,0.001102,-0.046221,0.021223,-0.033101,-0.035547,-0.036359,-0.025521,-0.030565
20645,zoo tour,-0.085650,-0.093741,0.015703,0.000577,-0.044756,-0.019462,0.033156,0.043889,-0.016135,...,-0.039359,-0.024103,-0.037887,-0.031642,0.035126,0.030659,0.034387,-0.029270,-0.023102,0.028840
20646,zoo yesterday,-0.000614,-0.019895,0.037936,0.041266,0.010509,-0.008908,-0.055139,0.003879,0.004546,...,0.015647,-0.014379,0.007880,0.053511,-0.051227,0.013682,-0.028846,0.012478,-0.002388,-0.062719
20647,zoological gardens,-0.085428,-0.061564,-0.039814,0.031094,-0.026854,-0.031232,0.040986,0.042299,-0.005840,...,0.024237,0.017969,-0.043686,-0.045863,0.056995,-0.008930,-0.053685,-0.084168,0.011695,0.033877


In [101]:
keywords_df = pd.read_excel('20200304_Aspects_Annotation.xlsx')
keywords_df = keywords_df[keywords_df['count'] >= 5]
keywords_df

,TEXT,count,SENTIMENT_KEYWORD_AGG_INDEX,cumulative,ASPECT_CATEGORY_NAME,ASPECT_CATEGORY,REQUIRE_CONTEXT
0,singapore,26872,1,26872,NaN,NaN,1.0
1,place,16418,2,43290,Sightseeing/ Place,NaN,NaN
2,gardens,15431,3,58721,Sightseeing/ Place,NaN,NaN
3,time,11895,4,70616,NaN,NaN,1.0
4,night,11791,5,82407,NaN,NaN,1.0
...,...,...,...,...,...,...,...
20642,zoo prices,5,20643,1105318,NaN,NaN,NaN
20643,zoo site,5,20644,1105323,NaN,NaN,NaN
20644,zoo tour,5,20645,1105328,NaN,NaN,NaN
20645,zoo yesterday,5,20646,1105333,NaN,NaN,NaN


In [102]:
parsed_col_2 = []
for keyword in list(keywords_df['TEXT']):
    keyword = str(keyword)
    keyword = keyword.replace('â€™', "'")
    keyword = keyword.replace('ã©', 'é')
    parsed_col_2.append(keyword)
keywords_df['TEXT'] = parsed_col_2

In [107]:
vectors_keywords_list = list(vectors_df[0])
ibm_keywords_list = list(keywords_df['TEXT'])

In [109]:
## Index of GloVe keywords not in IBM keyword list
for keyword in vectors_keywords_list:
    if keyword not in ibm_keywords_list:
        print(vectors_keywords_list.index(keyword))

11764
14975
15268
17312
17313
20648


In [110]:
## Index of IBM keywords not in GloVe keyword list
for keyword in ibm_keywords_list:
    if keyword not in vectors_keywords_list:
        print(ibm_keywords_list.index(keyword))

11765
15013
17371
17442


In [111]:
matrix = vectors_df.iloc[:, 1:].values
matrix.shape

(20649, 300)

In [113]:
def get_centroid(matrix):
    return np.mean(matrix, axis=0)

In [114]:
## mahatten, euclidean, cosine similarity, correlation

def get_manhattan(a, b):
    return np.abs(a - b).sum()

def get_euclidean(a, b):
    return np.linalg.norm(a-b)

def get_cos_sim(a, b):
    return np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))

def get_corr(a, b):
    return np.correlate(a, b)[0]

In [133]:
aspect_categories = list(set(keywords_df.ASPECT_CATEGORY_NAME))
aspect_categories = aspect_categories[1:]  # removing the nan category
aspect_categories.sort()
aspect_categories

['Activities',
 'Environment and Ambience',
 'Experience',
 'F&B',
 'Facilities',
 'Family/Children family/ Friends',
 'Price',
 'Service',
 'Shopping',
 'Sightseeing/ Place',
 'Transportation and Accessibility']

In [135]:
no_of_categories = len(aspect_categories)
centroids = np.zeros((no_of_categories, 300))

for i in range(no_of_categories):
    indices = keywords_df[keywords_df.ASPECT_CATEGORY_NAME == aspect_categories[i]].index
    centroids[i] = get_centroid(matrix[indices])
    
centroids

array([[ 0.34408881,  0.28954519,  0.14319013, ...,  0.29943544,
        -0.13584412,  0.01407281],
       [ 0.23309317,  0.20543953, -0.17164943, ...,  0.16502813,
         0.17355527,  0.1380384 ],
       [ 0.12541452,  0.14951679, -0.00992576, ...,  0.11087345,
         0.12158467,  0.04455952],
       ...,
       [ 0.63427837,  0.05797762,  0.51368575, ...,  0.21420537,
        -0.08110812,  0.11325012],
       [ 0.26318515,  0.16884861, -0.0960341 , ...,  0.14119422,
         0.07954345,  0.14238372],
       [ 0.29554432,  0.11429502,  0.05381467, ...,  0.21637387,
         0.05718621,  0.16618716]])

In [136]:
## Randomly generate category centroids from top 100

#top_n = np.copy(matrix[:no_of_categories*10, :])
#random.shuffle(top_n)
#clusters = np.split(top_n, no_of_categories)
#
#centroids = []
#for cluster in clusters:
#    centroids.append(get_centroid(cluster))
#    
#centroids = np.asarray(centroids)
#centroids.shape

In [137]:
manhattan_distances = np.zeros((matrix.shape[0], no_of_categories))
euclidean_distances = np.zeros((matrix.shape[0], no_of_categories))
cos_sim = np.zeros((matrix.shape[0], no_of_categories))
correlations = np.zeros((matrix.shape[0], no_of_categories))

def get_distances(centroids, matrix):
    for i in range(matrix.shape[0]):
        for j in range(no_of_categories):
            manhattan_distances[i, j] = get_manhattan(matrix[i], centroids[j])
            
    for i in range(matrix.shape[0]):
        for j in range(no_of_categories):
            euclidean_distances[i, j] = get_euclidean(matrix[i], centroids[j])
            
    for i in range(matrix.shape[0]):
        for j in range(no_of_categories):
            cos_sim[i, j] = get_cos_sim(matrix[i], centroids[j])
            
    for i in range(matrix.shape[0]):
        for j in range(no_of_categories):
            correlations[i, j] = get_corr(matrix[i], centroids[j])

In [138]:
%%time 
get_distances(centroids, matrix)

CPU times: user 8.6 s, sys: 16 ms, total: 8.61 s
Wall time: 8.6 s


In [143]:
def parse_to_df(matrix):
    final_df = pd.DataFrame(data=matrix)
    final_df['KEYWORD'] = vectors_df[0]
    cols = ['KEYWORD'] + [i for i in range(no_of_categories)]
    colnames = ['KEYWORD'] + [i for i in range(1, no_of_categories+1)]
    final_df = final_df.loc[:, cols]
    final_df.columns = colnames
    return final_df

euclidean_df = parse_to_df(euclidean_distances)
manhattan_df.head()

,KEYWORD,1,2,3,4,5,6,7,8,9,10,11
0,singapore,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,place,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,gardens,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,time,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,night,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
def get_closest_index(row):
    dictionary = dict(zip(row, list(range(1, no_of_categories+1))))
    return dictionary[min(row)]

manhattan_df['CLOSEST'] = manhattan_df.iloc[:,1:no_of_categories+1].apply(
    get_closest_index, 
    axis=1
)
manhattan_df.head()

,KEYWORD,1,2,3,4,5,6,7,8,9,10,11,CLOSEST
0,新加坡,92.317975,107.066505,127.263892,93.914772,120.379150,124.389095,121.400120,122.137160,123.086927,120.041290,123.501178,1
1,玩,89.030898,109.371380,106.632261,114.249332,106.228058,103.833656,109.387378,109.359585,111.494960,113.544791,96.025907,1
2,值得,84.545496,95.229495,99.719688,98.763281,85.958967,86.044239,95.039442,89.371342,97.230835,100.664753,96.042192,1
3,票,109.737668,101.596671,103.347272,109.140180,99.589848,95.660225,95.856264,97.404024,93.165697,108.537673,98.413517,9
4,时间,95.930932,99.897948,80.102679,94.631386,89.004047,98.097256,87.519456,96.233759,96.019486,103.597601,87.033281,3
